In [4]:
import requests
from lxml import etree
import pandas as pd

In [ ]:
# 初始化数据列表
df_list = []

# 循环遍历所有页面
page = 1
while page <= 52:
    url = f'https://www.sbgw.cn/ptzsb/school/{page}.html'
    try:
        response = requests.get(url)
        response.raise_for_status()  # 检查请求是否成功
        tree = etree.HTML(response.text)
        
        # 提取学校名称
        school_name = tree.xpath('//span[@href="#"]/text()')
        school_name = [name.strip() for name in school_name]  # 清理学校名称中的空白字符
        
        # 提取质量信息并清理
        quality = tree.xpath('//span[@class="lei_color"]/text()')
        cleaned_quality = [q.strip() for q in quality]
        
        # 提取年份信息并清理
        year1 = tree.xpath('//a[@class="collapsed"]/b/text()')
        cleaned_year1 = [year.strip() for year in year1]
        
        # 提取专业名称、原计划招生人数、调整后招生人数
        Professional_name = tree.xpath('//tbody/tr/td/a[1]/text()')
        number = tree.xpath('//tbody/tr/td[2]/b/text()')
        adj_number1 = tree.xpath('//tbody/tr/td[3]/b/text()')
        adj_number = [adj_numb.strip() for adj_numb in adj_number1]
        
        # 假设 quality 列表中第一个元素是性质，第二个元素是类别
        if len(cleaned_quality) >= 2:
            cleand_quality = cleaned_quality[0]
            cleand_cat = cleaned_quality[1]
        else:
            cleand_quality = None
            cleand_cat = None
        
        # 假设每个年份对应的专业数量相同
        num_years = len(cleaned_year1)
        num_professions = len(Professional_name) // num_years
        
        for i in range(num_years):
            year_info = {
                '年份': cleaned_year1[i],
                '招生专业': Professional_name[i * num_professions:(i + 1) * num_professions],
                '原计划招生人数': number[i * num_professions:(i + 1) * num_professions],
                '调整后招生人数': adj_number[i * num_professions:(i + 1) * num_professions]
            }
            
            for j in range(len(year_info['招生专业'])):
                df_list.append({
                    '院校名称': school_name[0] if school_name else None,
                    '类别': cleand_cat,
                    '性质': cleand_quality,
                    '年份': year_info['年份'],
                    '招生专业': year_info['招生专业'][j],
                    '原计划招生人数': year_info['原计划招生人数'][j],
                    '调整后招生人数': year_info['调整后招生人数'][j]
                })
    
    except requests.exceptions.RequestException as e:
        print(f"请求页面 {page} 时出错: {e}")
    except Exception as e:
        print(f"处理页面 {page} 时出错: {e}")
    
    page += 1

# 将数据转换为 DataFrame
df = pd.DataFrame(df_list)

# 保存 DataFrame 到 Excel 文件
df.to_excel('湖北专升本招生信息.xlsx', index=False)

print("数据已保存到湖北专升本招生信息.xlsx") 

In [43]:
df

,院校名称,类别,性质,年份,招生专业,原计划招生人数,调整后招生人数
0,长江大学专升本,综合类,公办,2024招生计划,视觉传达设计,103,103
1,长江大学专升本,综合类,公办,2024招生计划,产品设计,90,90
2,长江大学专升本,综合类,公办,2024招生计划,数字媒体艺术,160,160
3,长江大学专升本,综合类,公办,2024招生计划,广播电视编导,60,60
4,长江大学专升本,综合类,公办,2024招生计划,播音与主持艺术,60,60
...,...,...,...,...,...,...,...
3472,湖北工程学院新技术学院专升本,理工类,民办,2020招生计划,汉语言文学,15,25
3473,湖北工程学院新技术学院专升本,理工类,民办,2020招生计划,机械设计制造及其自动化,15,10
3474,湖北工程学院新技术学院专升本,理工类,民办,2020招生计划,计算机科学与技术,20,16
3475,湖北工程学院新技术学院专升本,理工类,民办,2020招生计划,土木工程,15,7


In [46]:
end = df[df['招生专业'] == '软件工程']
end

,院校名称,类别,性质,年份,招生专业,原计划招生人数,调整后招生人数
112,武汉纺织大学专升本,理工类,公办,2020招生计划,软件工程,23,74
513,湖北汽车工业学院专升本,理工类,公办,2020招生计划,软件工程,30,39
698,湖北工程学院专升本,理工类,公办,2024招生计划,软件工程,63,63
720,湖北工程学院专升本,理工类,公办,2023招生计划,软件工程,66,66
746,湖北工程学院专升本,理工类,公办,2022招生计划,软件工程,68,68
...,...,...,...,...,...,...,...
3166,武汉纺织大学外经贸学院专升本,财经类,民办,2020招生计划,软件工程,50,50
3185,湖北汽车工业学院科技学院专升本,理工类,民办,2021招生计划,软件工程,35,105
3433,湖北工程学院新技术学院专升本,理工类,民办,2024招生计划,软件工程,40,40
3449,湖北工程学院新技术学院专升本,理工类,民办,2022招生计划,软件工程,57,57


In [49]:
pub = end.groupby(['性质','年份'])


KeyboardInterrupt

